In [21]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [22]:
#Cargamos el archivo csv
df=pd.read_csv('Dallas_limpio.csv')
#Eliminamos la columna unnamed
df= df.drop(['Unnamed: 0'], axis=1)
df.head(20)

,last_scraped,source,description,neighborhood_overview,host_name,host_since,host_response_time,host_is_superhost,host_neighbourhood,host_verifications,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2024-11-18,city scrape,Close to downtown and Uptown. Fast and conven...,Enjoy the heart of Dallas right out your door ...,Sasha,2010-11-26,within an hour,f,Oak Lawn,"['email', 'phone']",...,4.62,4.85,4.92,4.75,4.77,1.0,1.0,0.0,0.0,0.33
1,2024-11-18,previous scrape,Additional discount for 2nd and 3rd month. Per...,"The address is 4340 Cedar Springs Road Dallas,...",Robert,2011-11-17,within an hour,f,Oak Lawn,"['email', 'phone', 'work_email']",...,4.87,4.91,4.96,4.89,4.93,1.0,1.0,0.0,0.0,0.40
2,2024-11-18,city scrape,***** Over 30% Discounts for stays over 30 day...,Located in Turtle Creek neighborhood steps fro...,Michelle,2012-11-19,within an hour,f,Oak Lawn,"['email', 'phone']",...,4.97,4.91,4.97,4.91,4.76,1.0,1.0,0.0,0.0,0.49
3,2024-11-18,city scrape,No description available,No overview available,Rodney,2011-07-11,a few days or more,f,North Central Dallas,"['email', 'phone']",...,4.74,4.85,4.85,4.81,4.78,1.0,0.0,0.0,0.0,0.24
4,2024-11-18,city scrape,"Traveling nurses, SMU students welcome - 10% d...",Wonderful big trees and nice sidewalks through...,Judy,2012-12-27,within an hour,t,Old Lake Highlands,"['email', 'phone']",...,4.82,4.93,4.93,4.92,4.83,3.0,0.0,0.0,0.0,0.72
5,2024-11-18,city scrape,"Welcome to the Knox District, a neighborhood t...","Great Community to Live, Work and Play<br />Lo...",Joan,2013-04-23,within an hour,f,Central Dallas,"['email', 'phone']",...,4.80,4.73,4.63,4.94,4.66,9.0,9.0,0.0,0.0,0.69
6,2024-11-18,city scrape,There are Two Suites - The Maxfield Parrish Su...,Very safe and friendly neighborhood to take mo...,C. F. Sandy,2013-06-13,within a few hours,f,North Central Dallas,"['email', 'phone', 'work_email']",...,4.80,4.93,4.93,4.50,4.53,1.0,0.0,0.0,0.0,0.33
7,2024-11-18,city scrape,Great location!! 6 min walk to train. 10 min w...,"The best neighborhood and the most desirable, ...",Holly,2012-10-09,within an hour,t,Lower Greenville,"['phone', 'work_email']",...,4.93,4.93,4.94,4.95,4.80,3.0,0.0,0.0,0.0,0.75
8,2024-11-18,city scrape,"Welcome to the Knox District, a neighborhood t...","Great Community to Live, Work and Play<br />Lo...",Joan,2013-04-23,within an hour,f,Central Dallas,"['email', 'phone']",...,4.66,4.92,4.78,4.86,4.53,9.0,9.0,0.0,0.0,0.54
9,2024-11-18,city scrape,Comfortable and private guest room with a quee...,The townhouse is conveniently located downtown...,Jeff,2013-10-21,within an hour,t,Downtown,"['email', 'phone']",...,4.97,4.97,4.98,4.90,4.92,3.0,1.0,0.0,0.0,4.85


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [24]:
#Verificar los valores sin repetirse de una columna
unico1 = np.unique(df['room_type'])
unico1

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [25]:
#Verificar los valores sin repetirse de una columna
unico2 = np.unique(df['bathrooms_text'])
unico2

array(['0 baths', '0 shared baths', '1 bath', '1 private bath',
       '1 shared bath', '1.5 baths', '1.5 shared baths', '11 baths',
       '12.5 baths', '14 baths', '2 baths', '2 shared baths', '2.5 baths',
       '2.5 shared baths', '3 baths', '3 shared baths', '3.5 baths',
       '4 baths', '4.5 baths', '5 baths', '5.5 baths', '6 baths',
       '6.5 baths', '7 baths', '8 baths', '9 baths', 'Half-bath',
       'Private half-bath', 'Shared half-bath'], dtype=object)

In [26]:
#Convierto una variable a dicotómica
df['room_type']= df['room_type'].replace(["Hotel room", "Private room", "Shared room"], "No Entire home/apt")
df['room_type']

0          Entire home/apt
1          Entire home/apt
2          Entire home/apt
3       No Entire home/apt
4       No Entire home/apt
               ...        
5302       Entire home/apt
5303       Entire home/apt
5304       Entire home/apt
5305       Entire home/apt
5306       Entire home/apt
Name: room_type, Length: 5307, dtype: object

In [27]:
# Extraer el número de baños como flotante
df["bathrooms_num"] = df["bathrooms_text"].str.extract(r'(\d+\.?\d*)').astype(float)

# Crear la variable dicotómica de manera automática
df["bathrooms_text"] = np.where(df["bathrooms_num"] > 1, ">1 baño", "≤1 baño")

# Verificar los cambios
df["bathrooms_text"].value_counts()


bathrooms_text
≤1 baño    3028
>1 baño    2279
Name: count, dtype: int64

In [28]:
#Convierto una variable a dicotómica
df['host_is_superhost']= df['host_is_superhost'].replace(["0"], "f")
df['host_is_superhost']

0       f
1       f
2       f
3       f
4       t
       ..
5302    t
5303    t
5304    f
5305    f
5306    f
Name: host_is_superhost, Length: 5307, dtype: object

In [29]:
#Seleccion de variables dependientes
Var_Dep1 = df["host_is_superhost"].unique()
print(Var_Dep1)
Var_Dep2 = df["instant_bookable"].unique()
print(Var_Dep2)
Var_Dep3 = df["host_identity_verified"].unique()
print(Var_Dep3)
Var_Dep4 = df["room_type"].unique()
print(Var_Dep4)
Var_Dep5 = df["bathrooms_text"].unique()
print(Var_Dep5)


['f' 't']
['f' 't']
['t' 'f']
['Entire home/apt' 'No Entire home/apt']
['≤1 baño' '>1 baño']


**Primer modelo de regresion logica.**

In [30]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep= df[['price']]
Var_Dep1= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep
y= Var_Dep1

In [31]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [32]:
#Se escalan todos los datos
escalar = StandardScaler()

In [33]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [34]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [35]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [36]:
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 't', ..., 'f', 't', 'f'], shape=(1593,), dtype=object)

In [37]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confunsion 1:')
print(matriz)

Matriz de Confunsion 1:
[[958  24]
 [582  29]]


In [38]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo 1:')
print(precision)

Precision del modelo 1:
0.6220779220779221


In [39]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo 1:')
print(exactitud)

Exactitud del modelo 1:
0.6195856873822976


In [40]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo 1:')
print(sensibilidad)

Sensibilidad del modelo 1:
0.04746317512274959


**Segundo modelo de regresion logistica.**

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [42]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep2= df[['review_scores_rating', 'review_scores_communication', 'review_scores_checkin']]
Var_Dep2= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep2
y= Var_Dep2

In [43]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [44]:
#Realizamos una prediccion
y_pred2 = algoritmo.predict(X_test)
y_pred2

array(['f', 't', 't', ..., 'f', 'f', 't'], shape=(1593,), dtype=object)

In [45]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred2)
print('Matriz de Confunsion 2:')
print(matriz)

Matriz de Confunsion 2:
[[732 238]
 [314 309]]


In [46]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred2, average="binary", pos_label="t")
print('Precision del modelo 2:')
print(precision)

Precision del modelo 2:
0.5648994515539305


In [47]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred2)
print('Exactitud del modelo 2:')
print(exactitud)

Exactitud del modelo 2:
0.6534839924670434


In [48]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred2, average="binary", pos_label="f")
print('Sensibilidad del modelo 2:')
print(sensibilidad)

Sensibilidad del modelo 2:
0.7546391752577319


**Tercer modelo de regresion logistica.**

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [50]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep3= df[['review_scores_rating', 'price', 'host_acceptance_rate']]
Var_Dep3= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep3
y= Var_Dep3

In [51]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [52]:
#Realizamos una prediccion
y_pred3 = algoritmo.predict(X_test)
y_pred3

array(['f', 'f', 'f', ..., 'f', 't', 'f'], shape=(1593,), dtype=object)

In [53]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred3)
print('Matriz de Confunsion 3:')
print(matriz)

Matriz de Confunsion 3:
[[896  64]
 [563  70]]


In [54]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred3, average="binary", pos_label="t")
print('Precision del modelo 3:')
print(precision)

Precision del modelo 3:
0.5223880597014925


In [55]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred3)
print('Exactitud del modelo 3:')
print(exactitud)

Exactitud del modelo 3:
0.6064030131826742


In [56]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred3, average="binary", pos_label="f")
print('Sensibilidad del modelo 3:')
print(sensibilidad)

Sensibilidad del modelo 3:
0.9333333333333333


**Cuarto modelo de regresion logistica.**

In [57]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep4= df[['number_of_reviews', 'price']]
Var_Dep4= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep4
y= Var_Dep4

In [58]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [59]:
#Realizamos una prediccion
y_pred4 = algoritmo.predict(X_test)
y_pred4

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(1593,), dtype=object)

In [60]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred4)
print('Matriz de Confunsion 4:')
print(matriz)

Matriz de Confunsion 4:
[[956   0]
 [637   0]]


In [61]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred4, average="binary", pos_label="f")
print('Precision del modelo 4:')
print(precision)

Precision del modelo 4:
0.6001255492780917


In [62]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred4)
print('Exactitud del modelo 4:')
print(exactitud)

Exactitud del modelo 4:
0.6001255492780917


In [63]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred4, average="binary", pos_label="t")
print('Sensibilidad del modelo 4:')
print(sensibilidad)

Sensibilidad del modelo 4:
0.0


**Quinto modelo de regresion logistica.**

In [64]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep5= df[['number_of_reviews', 'price', 'host_total_listings_count']]
Var_Dep5= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep5
y= Var_Dep5

In [65]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [66]:
#Realizamos una prediccion
y_pred5 = algoritmo.predict(X_test)
y_pred5

array(['t', 't', 't', ..., 't', 't', 't'], shape=(1593,), dtype=object)

In [67]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred5)
print('Matriz de Confunsion 5:')
print(matriz)

Matriz de Confunsion 5:
[[   0  124]
 [   0 1469]]


In [68]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred5, average="binary", pos_label="t")
print('Precision del modelo 5:')
print(precision)

Precision del modelo 5:
0.9221594475831764


In [69]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred5)
print('Exactitud del modelo 5:')
print(exactitud)

Exactitud del modelo 5:
0.9221594475831764


In [70]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred5, average="binary", pos_label="f")
print('Sensibilidad del modelo 5:')
print(sensibilidad)

Sensibilidad del modelo 5:
0.0


**Sexto modelo de regresion logistica.**

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [72]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep6= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep6= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep6
y= Var_Dep6

In [73]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [74]:
#Realizamos una prediccion
y_pred6 = algoritmo.predict(X_test)
y_pred6

array(['t', 't', 't', ..., 't', 't', 't'], shape=(1593,), dtype=object)

In [75]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred6)
print('Matriz de Confunsion 6:')
print(matriz)

Matriz de Confunsion 6:
[[   0  122]
 [   0 1471]]


In [76]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred6, average="binary", pos_label="t")
print('Precision del modelo 6:')
print(precision)

Precision del modelo 6:
0.9234149403640929


In [77]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred6)
print('Exactitud del modelo 6:')
print(exactitud)

Exactitud del modelo 6:
0.9234149403640929


In [78]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred6, average="binary", pos_label="f")
print('Sensibilidad del modelo 6:')
print(sensibilidad)

Sensibilidad del modelo 6:
0.0


**Septimo modelo de regresion logistica.**

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [80]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep7= df[['accommodates', 'host_acceptance_rate', 'price']]
Var_Dep7= df['room_type']

#Redefinimos las variables
X= Vars_Indep7
y= Var_Dep7

In [81]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [82]:
#Realizamos una prediccion
y_pred7 = algoritmo.predict(X_test)
y_pred7

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      shape=(1593,), dtype=object)

In [83]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred7)
print('Matriz de Confunsion 7:')
print(matriz)

Matriz de Confunsion 7:
[[1362    3]
 [ 156   72]]


In [84]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred7, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 7:')
print(precision)

Precision del modelo 7:
0.96


In [85]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred7)
print('Exactitud del modelo 7:')
print(exactitud)

Exactitud del modelo 7:
0.9001883239171374


In [86]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred7, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 7:')
print(sensibilidad)

Sensibilidad del modelo 7:
0.9978021978021978


**Octavo modelo de regresion logistica.**

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [88]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep8= df[['number_of_reviews', 'bedrooms']]
Var_Dep8= df['room_type']

#Redefinimos las variables
X= Vars_Indep8
y= Var_Dep8

In [89]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [90]:
#Realizamos una prediccion
y_pred8 = algoritmo.predict(X_test)
y_pred8

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      shape=(1593,), dtype=object)

In [91]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred8)
print('Matriz de Confunsion 8:')
print(matriz)

Matriz de Confunsion 8:
[[1383    8]
 [ 202    0]]


In [92]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred8, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 8:')
print(precision)

Precision del modelo 8:
0.0


In [93]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred8)
print('Exactitud del modelo 8:')
print(exactitud)

Exactitud del modelo 8:
0.8681732580037664


In [94]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred8, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 8:')
print(sensibilidad)

Sensibilidad del modelo 8:
0.9942487419122933


**Noveno modelo de regresion logistica.**

In [95]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep9= df[['number_of_reviews', 'price', 'accommodates']]
Var_Dep9= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep9
y= Var_Dep9

In [96]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [97]:
#Realizamos una prediccion
y_pred9 = algoritmo.predict(X_test)
y_pred9

array(['≤1 baño', '>1 baño', '≤1 baño', ..., '≤1 baño', '≤1 baño',
       '≤1 baño'], shape=(1593,), dtype=object)

In [98]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred9)
print('Matriz de Confunsion 9:')
print(matriz)

Matriz de Confunsion 9:
[[492 171]
 [ 99 831]]


In [99]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred9, average="binary", pos_label=">1 baño")
print('Precision del modelo 9:')
print(precision)

Precision del modelo 9:
0.8324873096446701


In [100]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred9)
print('Exactitud del modelo 9:')
print(exactitud)

Exactitud del modelo 9:
0.8305084745762712


In [101]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred9, average="binary", pos_label="≤1 baño")
print('Sensibilidad del modelo 9:')
print(sensibilidad)

Sensibilidad del modelo 9:
0.8935483870967742


**Decimo modelo de regresion logistica.**

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5307 entries, 0 to 5306
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  5307 non-null   object 
 1   source                                        5307 non-null   object 
 2   description                                   5307 non-null   object 
 3   neighborhood_overview                         5307 non-null   object 
 4   host_name                                     5307 non-null   object 
 5   host_since                                    5307 non-null   object 
 6   host_response_time                            5307 non-null   object 
 7   host_is_superhost                             5307 non-null   object 
 8   host_neighbourhood                            5307 non-null   object 
 9   host_verifications                            5307 non-null   o

In [103]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep10= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep10= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep10
y= Var_Dep10

In [104]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [105]:
#Realizamos una prediccion
y_pred10 = algoritmo.predict(X_test)
y_pred10

array(['≤1 baño', '≤1 baño', '≤1 baño', ..., '≤1 baño', '≤1 baño',
       '≤1 baño'], shape=(1593,), dtype=object)

In [106]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred10)
print('Matriz de Confunsion 10:')
print(matriz)

Matriz de Confunsion 10:
[[  0 663]
 [  0 930]]


In [107]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred10, average="binary", pos_label="≤1 baño")
print('Precision del modelo 10:')
print(precision)

Precision del modelo 10:
0.583804143126177


In [108]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred10)
print('Exactitud del modelo 10:')
print(exactitud)

Exactitud del modelo 10:
0.583804143126177


In [109]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred10, average="binary", pos_label=">1 baño")
print('Sensibilidad del modelo 10:')
print(sensibilidad)

Sensibilidad del modelo 10:
0.0
